# Google Forms API
This notebook is a documentation on the process of utilizing Google Forms API to retrieve end-user form responses and metadata.
<br/>

The following steps have been completed to ensure that proper credentials and authorizations have been set up on Google Cloud Platform.

1. Create a Google Cloud project for your Google Workspace app, extension, or integration.
2. Enable the APIs you want to use in your Google Cloud project.
3. Learn how authentication and authorization works when developing for Google Workspace.
4. Configure OAuth consent to ensure users can understand and approve what access your app has to their data.
5. Create access credentials to authenticate your app's end users or service accounts.

### Table of Content
1. Connecting to Google Forms API and retrieve the latest response and metadata
2. Extracting values from nested dictionaries
3. Joining the response table with the metadata table
4. Concatenating the relevant information

#### 1. Connecting to Google Forms API and retrieve the latest response and metadata

In [1]:
from __future__ import print_function
from googleapiclient import discovery
from apiclient import discovery
from httplib2 import Http
from oauth2client import client, file, tools

In [2]:
SCOPES = "https://www.googleapis.com/auth/forms.responses.readonly"
DISCOVERY_DOC = "https://forms.googleapis.com/$discovery/rest?version=v1"

store = file.Storage('token.json')
creds = None
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_secrets.json', SCOPES)
    flags = tools.argparser.parse_args(args=[])
    creds = tools.run_flow(flow, store, flags)
    service = discovery.build('forms', 'v1', http=creds.authorize(
    Http()), discoveryServiceUrl=DISCOVERY_DOC, static_discovery=False)
    
# Prints the responses of your specified form:
form_id = '1g62TUlQuijz5fl-DLDftzsWWy5Q4hELMJwrHIHwj3B4'
result = service.forms().responses().list(formId=form_id).execute()


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=639527455761-80cjs18la5619rc16mt4mkotvr34308k.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fforms.responses.readonly&access_type=offline&response_type=code

If your browser is on a different machine then exit and re-run this
application with the command-line parameter

  --noauth_local_webserver

Authentication successful.


In [3]:
SCOPES = "https://www.googleapis.com/auth/forms.body.readonly"
DISCOVERY_DOC = "https://forms.googleapis.com/$discovery/rest?version=v1"

store = file.Storage('token.json')
creds = None
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_secrets.json', SCOPES)
    flags = tools.argparser.parse_args(args=[])
    creds = tools.run_flow(flow, store, flags)
    service = discovery.build('forms', 'v1', http=creds.authorize(
    Http()), discoveryServiceUrl=DISCOVERY_DOC, static_discovery=False)

# Prints the title of the sample form:
form_id = '1g62TUlQuijz5fl-DLDftzsWWy5Q4hELMJwrHIHwj3B4'
result2 = service.forms().get(formId=form_id).execute()


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=639527455761-80cjs18la5619rc16mt4mkotvr34308k.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fforms.body.readonly&access_type=offline&response_type=code

If your browser is on a different machine then exit and re-run this
application with the command-line parameter

  --noauth_local_webserver

Authentication successful.


#### 2. Extracting values from nested dictionaries

The response and metadata downloaded have dictionaries nested within lists. We will therefore be using for loops to extract the necessary information. During this process, we will also be inserting a new column to translate the Chinese characters into English.

In [4]:
import pandas as pd
import numpy as np

In [5]:
name_eng = ["Coke Zero", "Coke Original", "Pokka Green Tea", "Pokka Oolong Tea", "China Apple","Season Ice Lemon Tea"
            , "RedBull (Gold)","Elements Premium Drinking Water 500ml", "Elements Premium Drinking Water 1.5L", 
            "Yeos Winter Melon Tea", "Yeos Lemon Barley","Kickapoo", "Pepsi", "Groovy Grape", "HSC Oldenlandia", 
            "100 Plus", "Juice Secret Pineapple", "Yeos Coconut Juice Drink", "Jia Jia Herbal",
            "Pokka Green Tea 1.5L", "Seasons Ice Lemon Tea 1.5L", "100 Plus 1.5L", "Planta Butter"
            ,"Marigold Evaporated Creamer" ,"Kaya"]

In [6]:
drinksid = []
name = []
for i in result2['items']:
    drinksid.append(i['questionItem']['question']['questionId'])
    name.append((i['title']))
drinks_info = {'drinksid': drinksid, 'name':name, 'name_eng':name_eng}
drinks_info = pd.DataFrame(drinks_info)
drinks_info.head()

,drinksid,name,name_eng
0,1ba9a5f0,可口可乐 (黑),Coke Zero
1,6bdb79dc,可口可乐 (红),Coke Original
2,359eb913,绿茶,Pokka Green Tea
3,01ff9043,乌龙茶,Pokka Oolong Tea
4,781aa0d5,中国苹果,China Apple


In [7]:
my_list = result['responses'][0]['answers'].values()

In [8]:
drinksid = []
quantity = []
for i in my_list:
    quantity.append((list(i['textAnswers']['answers'])[0]['value']))
    drinksid.append(list(i.values())[0])
drinks_order = {'drinksid':drinksid, 'quantity':quantity}
drinks_order = pd.DataFrame(drinks_order)
drinks_order.head()

,drinksid,quantity
0,08b92b62,1
1,01ff9043,1
2,07be32c0,12
3,4e20c475,1
4,30e76a3a,1


#### 3. Joining the response table with the metadata table

In [9]:
drinks_order_final = drinks_order.merge(drinks_info[['drinksid','name_eng']], on='drinksid', how='left')

In [10]:
drinks_order_final.head()

,drinksid,quantity,name_eng
0,08b92b62,1,HSC Oldenlandia
1,01ff9043,1,Pokka Oolong Tea
2,07be32c0,12,Kaya
3,4e20c475,1,Yeos Lemon Barley
4,30e76a3a,1,Season Ice Lemon Tea


#### 4. Concatenating the relevant information

In [11]:
finalorder = drinks_order_final["name_eng"] + " " + drinks_order_final["quantity"]

In [12]:
finalorder.to_csv('orderfinal.txt',index=False)